In [40]:
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn import svm
from sklearn.linear_model import Perceptron
import numpy as np
import pandas as pd

In [41]:
def mensure(X,y,classifier):
    X_train, X_test, y_train, y_test = train_test_split(X, y, random_state=0)
    y_pred = classifier.fit(X_train, y_train).predict(X_test)
    ac = accuracy_score(y_test, y_pred)
    
    return ac

def predict(X,y,values):
    test = np.array(values)
    y_pred = classifier.fit(X, y).predict(test)
    
    return y_pred

In [48]:
if __name__ == '__main__':
    
    data = load_iris()
    iris_df = pd.DataFrame()
    X = data.data
    y = data.target
    class_names = data.target_names
    classifier = svm.SVC(kernel='linear', C=100)
    ac = mensure(X,y,classifier)
    cl = Perceptron(n_iter=150)
    ac2 = mensure(X,y,cl)
    print("SVM ac = %f"%ac)
    print("Perceptron ac = %f"%ac2)

SVM ac = 0.947368
Perceptron ac = 0.947368
